In [1]:
import numpy as np
import scipy as sc
import roboticstoolbox as rtb
import spatialmath as sm
import matplotlib.pyplot as plt
from scipy.optimize import fsolve


In [98]:
class irb140_clase(rtb.DHRobot):
    def __init__(self):
        super().__init__([
            rtb.RevoluteDH(alpha=-np.pi/2,a=0.07, qlim=[-np.pi,np.pi]),
            rtb.RevoluteDH(a=0.36,offset=-np.pi/2, qlim=[-np.pi,np.pi]),
            rtb.RevoluteDH(alpha=np.pi/2,offset=np.pi, qlim=[-np.pi,np.pi]),
            rtb.RevoluteDH(d=0.38, alpha=-np.pi/2, qlim=[-np.pi,np.pi]),
            rtb.RevoluteDH(alpha=np.pi/2, qlim=[-np.pi,np.pi]),
            rtb.RevoluteDH()
        ], name="IRB140")

    def get_config(self,q):
        g1 = np.sign(self.links[3].d * np.sin(q[1]+self.links[1].offset +q[2]+self.links[2].offset) + self.links[1].a * np.cos(q[1]+self.links[1].offset) + self.links[0].a)
        g2 = np.sign(np.cos(q[2]+self.links[2].offset))
        g3 = np.sign(np.sin(q[4]+self.links[4].offset))
        return np.array([g1,g2,g3])

    def get_joint_bounds(self):
        return np.array([link.qlim if link.qlim is not None else [-np.pi, np.pi] for link in self.links])

    def solve_q1(self, g1, q2):
        d3, o1, o2, a1, a0 = self.links[3].d, self.links[1].offset, self.links[2].offset, self.links[1].a, self.links[0].a
        def S(theta): return d3 * np.sin(theta + q2 + o2) + a1 * np.cos(theta) + a0
        sol = fsolve(S, 0.0)[0]
        theta = sol + np.pi/6 if g1 * S(sol + np.pi/6) > 0 else sol - np.pi/6
        return theta - o1

    def guess_seed_from_conf(self, conf,POSE ,q_ref=None):
        q0 = np.zeros(6)

        px, py, pz = POSE.t
        g1 = conf[0]
        if px >= 0 and py >= 0: 
            q0[0] = np.pi/4 if g1 == 1 else -3*np.pi/4
        elif px >= 0 and py < 0:  
            q0[0] = -np.pi/4 if g1 == 1 else 3*np.pi/4
        elif px < 0 and py >= 0:  
            q0[0] = 3*np.pi/4 if g1 == 1 else -np.pi/4
        else:  
            q0[0] = -3*np.pi/4 if g1 == 1 else np.pi/4

        q0[2] = np.pi/2 + conf[1] * np.pi/3
        q0[4] = conf[2] * np.pi/4
        q0[1] = self.solve_q1(conf[0], q0[2])
        return 0.5*q0 + 0.5*q_ref if q_ref is not None else q0

    def wrap_joints(self, q):
        q_wrapped = np.copy(q)
        bounds = self.get_joint_bounds()
        for i in range(len(q)):
            q_min, q_max = bounds[i]
            span = q_max - q_min
            q_wrapped[i] = (q[i] - q_min) % span + q_min
        return q_wrapped

    def damped_pinv(self, J, damping=1e-4):
        U, S, Vh = np.linalg.svd(J)
        S_inv = S / (S**2 + damping**2)
        return Vh.T @ np.diag(S_inv) @ U.T

    def config_match(self, q, conf):
        return np.all(self.get_config(q) == conf)

    def ikine_recta_con_trayectoria(self, POSE_deseada, conf, q_inicial=None,
                                    vd=0.01, Ts=0.001, tol=1e-3, max_iter=2000,
                                    q_actual=None):
        q_i = q_inicial if q_inicial is not None else (
            q_actual.copy() if q_actual is not None else self.guess_seed_from_conf(conf, POSE_deseada)
        )
        trayectoria = [q_i]
        errores = []

        for i in range(max_iter):
            POSE_i = self.fkine(q_i)
            e_vec = sm.base.tr2delta(POSE_i.A, POSE_deseada.A)
            error_norm = np.linalg.norm(e_vec)
            errores.append(error_norm)

            if error_norm < tol:
                return (q_i, 0, np.array(trayectoria), errores) if self.config_match(q_i, conf) else (q_i, -2, np.array(trayectoria), errores)

            delta_pose_i = (e_vec / error_norm) * vd * Ts
            J = self.jacobe(q_i)
            dq_i = self.damped_pinv(J) @ delta_pose_i
            q_i = self.wrap_joints(q_i + dq_i)
            trayectoria.append(q_i)

        return q_i, -1, np.array(trayectoria), errores

    def ikine_recta_con_trayectoria_multi(self, POSE_deseada, conf, q_actual=None,
                                          vd=0.01, Ts=0.001, tol=1e-3, max_iter=2000,
                                          n_reintentos=6):
        for intento in range(n_reintentos):
            if intento == 0 and q_actual is not None:
                q_seed = q_actual
            elif intento == 1:
                q_seed = self.guess_seed_from_conf(conf, POSE_deseada)
            else:
                perturb = (np.random.rand(6)-0.5)*0.2
                q_seed = self.guess_seed_from_conf(conf, POSE_deseada) + perturb

            q, status, trayectoria, errores = self.ikine_recta_con_trayectoria(
                POSE_deseada, conf, q_inicial=q_seed, vd=vd, Ts=Ts, tol=tol, max_iter=max_iter
            )
            if status == 0:
                return q, status, trayectoria, errores
        return q, status, trayectoria, errores
    

In [99]:
robot = irb140_clase()
n_acierto = 0
n_iter = 1

for i in range(n_iter):
    q_deseado = (np.random.rand(6)-0.5)*2*np.pi
    POSE = robot.fkine(q_deseado)
    conf = robot.get_config(q_deseado)
    q, success, history, _ = robot.ikine_recta_con_trayectoria_multi(
        POSE, conf=conf, vd=1, Ts=0.001, max_iter=8000, n_reintentos=10
    )
    if success == 0:
        n_acierto += 1
    else:
        print(f"❌ ERROR:\nDeseado: {q_deseado}\nConfig: {conf}\nAlcanzado: {q}")

print("✅ Cantidad de corridas:", n_iter)
print("🎯 Cantidad de aciertos:", n_acierto)


✅ Cantidad de corridas: 1
🎯 Cantidad de aciertos: 1


In [148]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import spatialmath as sm
import roboticstoolbox as rtb

# --- Paso 1: Generar datos ---
def generar_dataset(robot, n_samples=100000):
    qs = np.random.uniform(0, 2*np.pi, (n_samples, 6))
    poses = []
    confs = []

    for q in qs:
        T = robot.fkine(q)
        pose = np.hstack((T.t, T.R.flatten()))
        conf = robot.get_config(q)
        poses.append(np.hstack((pose, conf)))

    X = np.array(poses, dtype=np.float32)
    y = np.array(qs, dtype=np.float32)
    return X, y

# --- Paso 2: Definir la red ---
class IKNet(nn.Module):
    def __init__(self, input_dim=15, joint_bounds=None, hidden=256):
        super().__init__()
        self.joint_bounds = torch.tensor(joint_bounds, dtype=torch.float32)
        self.net = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 6)
        )

    def forward(self, x):
        y = self.net(x)
        return y

# --- Paso 3: Entrenamiento ---
def entrenar_red(robot, epochs=300, batch_size=1024):
    X, y = generar_dataset(robot)
    X_train = torch.tensor(X)
    y_train = torch.tensor(y)

    dataset = torch.utils.data.TensorDataset(X_train, y_train)
    loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

    model = IKNet(input_dim=X.shape[1], joint_bounds=robot.get_joint_bounds())
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    loss_fn = nn.MSELoss()

    for epoch in range(epochs):
        total_loss = 0
        for xb, yb in loader:
            pred = model(xb)
            loss = loss_fn(pred, yb)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}/{epochs} - Loss: {total_loss/len(loader):.6f}")

    return model

# --- Paso 4: Evaluación ---
def evaluar(model, robot, n_test=100):
    X_test, y_test = generar_dataset(robot, n_samples=n_test)
    X_t = torch.tensor(X_test)
    y_pred = model(X_t).detach().numpy()

    errores = np.abs((y_pred - y_test + np.pi) % (2*np.pi) - np.pi)
    mae = np.mean(errores)
    print(f"Error medio absoluto: {mae:.4f} rad")
    return mae

# --- Uso ---
if __name__ == "__main__":
    robot = irb140_clase()
    model = entrenar_red(robot, epochs=300)
    evaluar(model, robot)



Epoch 1/300 - Loss: 3.997307
Epoch 2/300 - Loss: 2.668416
Epoch 3/300 - Loss: 2.458839
Epoch 4/300 - Loss: 2.211545
Epoch 5/300 - Loss: 1.947850
Epoch 6/300 - Loss: 1.799505
Epoch 7/300 - Loss: 1.661745
Epoch 8/300 - Loss: 1.555153
Epoch 9/300 - Loss: 1.472501
Epoch 10/300 - Loss: 1.407245
Epoch 11/300 - Loss: 1.328690
Epoch 12/300 - Loss: 1.248085
Epoch 13/300 - Loss: 1.178978
Epoch 14/300 - Loss: 1.134945
Epoch 15/300 - Loss: 1.099222
Epoch 16/300 - Loss: 1.071633
Epoch 17/300 - Loss: 1.041620
Epoch 18/300 - Loss: 1.018390
Epoch 19/300 - Loss: 0.994287
Epoch 20/300 - Loss: 0.974190
Epoch 21/300 - Loss: 0.952761
Epoch 22/300 - Loss: 0.934326
Epoch 23/300 - Loss: 0.914403
Epoch 24/300 - Loss: 0.902552
Epoch 25/300 - Loss: 0.888559
Epoch 26/300 - Loss: 0.875113
Epoch 27/300 - Loss: 0.863425
Epoch 28/300 - Loss: 0.857809
Epoch 29/300 - Loss: 0.843561
Epoch 30/300 - Loss: 0.837095
Epoch 31/300 - Loss: 0.827196
Epoch 32/300 - Loss: 0.818379
Epoch 33/300 - Loss: 0.810406
Epoch 34/300 - Loss

In [155]:
q_deseado = (np.random.rand(6)-0.5)*2*np.pi
POSE = robot.fkine(q_deseado)
conf = robot.get_config(q_deseado)
pose = np.hstack((POSE.t, POSE.R.flatten()))
x = np.hstack((pose, conf))
y = model(torch.tensor(x, dtype=torch.float32)).detach().numpy()
print("Deseado:", q_deseado)
print("Predicho:", y)


Deseado: [ 2.68344555 -3.11645747  1.07200485  0.73256333 -1.031064    0.13795487]
Predicho: [2.802371   3.1434662  1.0736734  0.57491785 5.1163034  1.8825862 ]
